In [1]:
import pandas as pd
import sqlite3
import seaborn as sns
import numpy as np


In [2]:
import os
os.getcwd()

'c:\\Users\\user\\Documents\\Phase-2-Movie-Analysis-Project\\notebooks'

In [3]:
os.listdir()

['Daniella.ipynb',
 'data_cleaning.ipynb',
 'Eve.ipynb',
 'final_notebook.ipynb',
 'Jack.ipynb',
 'Naomi .ipynb',
 'Verah.ipynb']

In [6]:
rt_info = pd.read_csv("../data/zippedData/rt.movie_info.tsv.gz", sep="\t", compression="gzip")
rt_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [13]:
conn = sqlite3.connect('rottentomatoes.db')


In [15]:
rt_info = pd.read_csv("../data/zippedData/rt.movie_info.tsv.gz", sep="\t", compression="gzip"
)
rt_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [16]:
rt_info.to_sql('rt_info', conn, if_exists='replace', index=False)

1560

In [17]:
query = "SELECT * FROM rt_info LIMIT 10;"
result = pd.read_sql_query(query, conn)
print(result)

   id                                           synopsis rating  \
0   1  This gritty, fast-paced, and innovative police...      R   
1   3  New York City, not-too-distant-future: Eric Pa...      R   
2   5  Illeana Douglas delivers a superb performance ...      R   
3   6  Michael Douglas runs afoul of a treacherous su...      R   
4   7                                               None     NR   
5   8  The year is 1942. As the Allies unite overseas...     PG   
6  10  Some cast and crew from NBC's highly acclaimed...  PG-13   
7  13  Stewart Kane, an Irishman living in the Austra...      R   
8  14  "Love Ranch" is a bittersweet love story that ...      R   
9  15  When a diamond expedition in the Congo is lost...  PG-13   

                                               genre          director  \
0                Action and Adventure|Classics|Drama  William Friedkin   
1                  Drama|Science Fiction and Fantasy  David Cronenberg   
2                  Drama|Musical and Per

In [20]:
query = """
SELECT "writer","director", COUNT(*) AS cnt 
FROM rt_info GROUP BY "writer","director" 
HAVING cnt > 1;"""
duplicates = pd.read_sql_query(query, conn)
print(duplicates)
print("Duplicate rows before cleanup", len(duplicates))

                                        writer             director  cnt
0                                         None                 None  131
1                                         None         Billy Wilder    2
2                                         None          Boris Sagal    2
3                                         None           Don Siegel    2
4                                         None       Gordon Douglas    2
5                                         None        James Cameron    2
6                                         None       Jonathan Demme    2
7                                         None        Lance Comfort    2
8                                         None          Martin Ritt    2
9                                         None         Richard Lang    2
10                                        None          Sam Weisman    2
11                                        None     Steven Spielberg    2
12                                        None     

In [21]:
query = """
DELETE FROM rt_info
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM rt_info
    GROUP BY "writer", "director"
);"""
conn.execute(query)
conn.commit()

In [22]:
query = """
SELECT "writer","director", COUNT(*) AS cnt
FROM rt_info GROUP BY "writer","director"
HAVING cnt > 1;"""
duplicates_after_cleanup = pd.read_sql_query(query, conn)
print(duplicates_after_cleanup)
print("Duplicate rows after cleanup", len(duplicates_after_cleanup))

Empty DataFrame
Columns: [writer, director, cnt]
Index: []
Duplicate rows after cleanup 0


In [23]:
query = "SELECT COUNT(*) AS total_rows FROM rt_info;"
total_rows = pd.read_sql_query(query, conn)
print("Total rows in rt_info after cleanup:", total_rows['total_rows'][0])

Total rows in rt_info after cleanup: 1393


In [32]:
query = """
SELECT CAST("runtime" AS INTEGER) AS runtime_int
FROM rt_info
WHERE "runtime" IS NOT NULL AND "runtime" != ''
LIMIT 10;"""
pd.read_sql_query(query, conn)




,runtime_int
0,104
1,108
2,116
3,128
4,200
5,95
6,82
7,123
8,117
9,108


In [31]:
query = """
SELECT CAST(box_office AS INTEGER) AS box_office_int
FROM rt_info
WHERE box_office IS NOT NULL AND box_office != ''
LIMIT 10;"""
pd.read_sql_query(query, conn)

,box_office_int
0,600
1,41
2,224
3,134
4,1
5,20
6,1
7,312
8,201
9,54


In [35]:
rt_reviews= pd.read_csv("../data/zippedData/rt.reviews.tsv.gz", sep="\t", compression="gzip", chunksize=100_000)


In [38]:
rt_reviews = pd.read_csv(
    "../data/zippedData/rt.reviews.tsv.gz", 
      sep="\t",
      compression="gzip",
      chunksize=100_000,
      encoding="utf-8",
      on_bad_lines="skip"
)

In [41]:
rt_review_iter = pd.read_csv(
    "../data/zippedData/rt.reviews.tsv.gz", 
      sep="\t",
      compression="gzip",
      chunksize=100_000,
      encoding="latin1",
      on_bad_lines="skip"
)

In [43]:
for chunk in rt_review_iter:
    chunk.to_sql('rt_reviews', conn, if_exists='append', index=False)

In [44]:
pd.read_sql_query("SELECT COUNT(*) AS review_count FROM rt_reviews;", conn)

,review_count
0,54432


In [45]:
querry = """
SELECT * FROM rt_reviews LIMIT 10;"""
pd.read_sql_query(querry, conn)

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,None,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,None,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,None,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,None,fresh,None,0,Cinema Scope,"October 12, 2017"
5,3,... Cronenberg's Cosmopolis expresses somethin...,None,fresh,Michelle Orange,0,Capital New York,"September 11, 2017"
6,3,"Quickly grows repetitive and tiresome, meander...",C,rotten,Eric D. Snider,0,EricDSnider.com,"July 17, 2013"
7,3,Cronenberg is not a director to be daunted by ...,2/5,rotten,Matt Kelemen,0,Las Vegas CityLife,"April 21, 2013"
8,3,"Cronenberg's cold, exacting precision and emot...",None,fresh,Sean Axmaker,0,Parallax View,"March 24, 2013"
9,3,Over and above its topical urgency or the bit ...,None,fresh,Kong Rithdee,0,Bangkok Post,"March 4, 2013"


In [46]:
querry = """DELETE FROM rt_reviews
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM rt_reviews
    GROUP by review, publisher, review, rating, date, fresh

);"""
conn.execute(querry)
conn.commit()

In [54]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(":memory:")

pd.read_sql_query(
    "SELECT 1 AS test_column;",
    conn
)


,test_column
0,1
